# sudregex Tutorial   

In this tutorial, we will walk through the process of using `sudregex` package.
The goal of this tutorial is to illustrate the basic components and concepts of sudregex in a simple way. 

In [6]:
#importing neccessary packages 
import pandas as pd   
import sudregex as sud
import warnings
warnings.filterwarnings('ignore')

# 1. Loading Data 

Load clinical notes

In [ ]:
#Directory of all the files needed for the tutorial
dir= 'your_directory_here'  #Change this to your directory
source_dirs = {
    'tutorial_data': dir + '/input_data.txt',
    'checklist_file': dir + '/your_checklist.py',
    'termlist_file': dir + '/termslist.py',
    'output_data': dir + '/output_data.txt'
}

In [ ]:
## Access the file paths from the dictionary
tutorial_data = source_dirs['tutorial_data']
checklist_file = source_dirs['checklist_file']
termlist_file = source_dirs['termlist_file']
output_data = source_dirs['output_data']

In [ ]:
#use pandas to read in the clinical notes
df_notes= pd.read_csv(tutorial_data,sep=r' !\^! ', header=None, engine='python')
df_notes.head(3)

,0,1,2
0,R3000000025,800000000000025,"""Patient smokes half a pack of cigarettes daily""."
1,R3000000026,800000000000026,"""Denies any current use of illicit substances."""
2,R3000000027,800000000000027,"""Discharge instructions include avoiding alcoh..."


In [16]:
# Renaming the columns for better readability
df_notes.columns = ["patient_id", "note_id", "note_text"]
df_notes.head(3)

,patient_id,note_id,note_text
0,R3000000025,800000000000025,"""Patient smokes half a pack of cigarettes daily""."
1,R3000000026,800000000000026,"""Denies any current use of illicit substances."""
2,R3000000027,800000000000027,"""Discharge instructions include avoiding alcoh..."


 ## The Core Function of `sudregex`: `extract_df`

The heart of the `sudregex` package is the function **`extract_df()`**, which applies predefined
regular expression (regex) rules to clinical note text.

### Arguments to specify 

**1) Checklist**
- Think of this as a **rulebook of keywords/patterns** to flag in text.
- By default, `sudregex` provides the **ABC (Addiction Behavior Checklist)**, which includes regex
  rules to identify addiction-related patterns (e.g., “substance craving,” “missed appointments,”
  “uncontrolled use”).
- You can also **load your own checklist** to adapt the package to your research context.

**2) Termslist**
- A **dictionary of substance-related keywords and phrases**, grouped into categories.
- Default term groups include:
  - **Alcohol**
  - **Opioids**
  - **Chronic Pain–related terms**
- Each group can be **activated independently** (e.g., only search for opioids) or **combined**,
  depending on your analysis needs.

### ⚙️ How It All Fits Together

When you pass a DataFrame of clinical notes to `extract_df`:
- The **termslist** helps locate mentions of specific substances (e.g., “opioids”) to ensure pattern matches are specific to substance of interest.
- The **checklist** applies higher-level rules that capture **behavioral or contextual signals** of
  substance use (e.g., “denies use,” “positive screen,” “discharge instructions”).
- The result is a **structured DataFrame** with new **indicator columns** that flag these matches,
  ready for **downstream analysis** (e.g., prevalence estimates, modeling features).


In [ ]:
result = sud.extract_df(
    df_notes,
    checklist = checklist_file,  # you could do path-style here too if you wanted
    termslist = termlist_file,  # you could do path-style here too if you wanted
    terms_active = "opioid_terms", #active termlist to use
    note_column = "note_text", # name of the column with the text to be processed
    debug = False,  # set to True to see debug output
     # set to True to see debug output
)
result.head(3)

,note_id,illicit_drugs,illicit_drugs_SUBSTANCE_MATCHED,illicit_drugs_SUBSTANCE_MATCHED_NEG,problem_drinking,problem_drinking_NEG,dui,dui_NEG,hoarding,hoarding_SUBSTANCE_MATCHED,...,lack_interest_rehab_SUBSTANCE_MATCHED,minimal_relief_x,minimal_relief_x_SUBSTANCE_MATCHED,tolerance,tolerance_SUBSTANCE_MATCHED,tolerance_SUBSTANCE_MATCHED_NEG,med_agreement,SO_concern,SO_concern_SUBSTANCE_MATCHED,SO_concern_SUBSTANCE_MATCHED_NEG
0,800000000000025,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,800000000000026,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,800000000000027,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Disambiguation & Context Filters -- Optional

Short and sweet: even good regex will catch **spurious matches** (e.g., “family history” vs. true history-of-use)
and **template text** (e.g., discharge instructions) that can distort your counts. `sudregex` includes helpers to
reduce these errors.

### 1) Filter Common False Positives
Use `check_common_false_positives` to *zero-out* flags when a term appears in a known **non-informative phrase**.


In [5]:
#check for common false positives in your data

# pat, df_searched, col_name_fp, common_fp, span=20 -- add this arguments 
import re
df = pd.DataFrame(
        {"note_id": [1],
         "note_text": ["family history of asthma"], 
         "history": [1]}
    )

df_out = sud.check_common_false_positives(
        re.compile("history"), #
        df.copy(), 
        "history", 
        ["family"]
    )
    # 'family history' should be filtered, so hist → 0

df_out

,note_id,note_text,history
0,1,family history of asthma,0


## 2) Remove “Discharge Instructions” Context (Optional)

Use discharge_instructions to clear flags within discharge sections, which are often templated/repeated.

In [ ]:

df = pd.DataFrame({
    "note_id": [1, 2],
    "note_text": ["Discharge instructions: avoid alcohol. foo", "No discharge section. foo"],
    "foo": [1, 1]   # example flag from earlier regex
})

# Provide the same pattern you previously used to set `foo`
out2 = sud.discharge_instructions(
    re.compile(r"foo", 
               flags=re.I),
    df.copy(),
    "foo"
)

# Expect: row 1 -> foo=0 (discharge context), row 2 -> foo=1 (kept)
out2

,note_id,note_text,foo
0,1,Discharge instructions: avoid alcohol. foo,0
1,2,No discharge section. foo,1


## ✏️ Customizing Checklists and Termslists (Notebook-Friendly)

One strength of `sudregex` is that you don’t have to rely only on the defaults —
you can **append new terms and rules** inside your notebook.  
This lets you experiment without permanently editing the package files.



In [ ]:
import sudregex as sud

# Load the termslist (module or dict depending on your build)
raw_terms = sud.default_termslist

# If it's a module, convert to a dict of lists; if it's already a dict, keep as-is.
if hasattr(raw_terms, "__dict__") and not isinstance(raw_terms, dict):
    termslist = {
        k: getattr(raw_terms, k)
        for k in dir(raw_terms)
        if not k.startswith("_") and isinstance(getattr(raw_terms, k), (list, tuple))
    }
else:
    termslist = dict(raw_terms)  # shallow copy so you can edit safely

print("Available groups:", sorted(termslist.keys()))


Available groups: ['alcohol_terms', 'chronic_pain_terms', 'opioid_terms']


In [8]:

# Add new terms to an existing group
termslist['opioid_terms'].extend(['new added', 'new addded 2'])

# Confirm the update
print("Updated opioid terms:", termslist['opioid_terms'])


Updated opioid terms: ['pain med', 'opioid', 'opiod', 'narc', 'analges', 'suboxone', 'Avinza', 'codeine', 'dilaudid', 'fentanyl', 'hydrocodone', 'morphine', 'opana', 'opiate', 'oxycodone', 'oxycontin', 'oxymorphone', 'percocet', 'roxicodone', 'sufentanyl', 'vicodin', 'lortab', 'hydromorphone', 'abstral', 'actiq', 'alfentanil', 'arymo', 'ascomp', 'astramorph', 'avinza', 'belbuca', 'brompheniramine', 'bunavail', 'buprenex', 'buprenorphine', 'butalbital', 'butorphanol', 'butrans', 'capcof', 'carisoprodol', 'cheratussin', 'coditussin', 'conzip', 'demerol', 'dexbrompheniramine', 'dihydrocodeine', 'diskets', 'dolophine', 'durmorph', 'embeda', 'endacof', 'endocet', 'exalgo', 'fentora', 'fioricet', 'flowtuss', 'guaifenesin', 'histex', 'hycet', 'hycofenix', 'hydrocodone', 'hydromorphone', 'hysingla', 'ibudone', 'infumorph', 'iophen', 'iorinal', 'kadian', 'lazanda', 'levorphanol', 'lorcet', 'lotruss', 'meperidine', 'methadone', 'methadose', 'morphabond', 'morphine', 'ms contin', 'nalbuphine', 'n

## DataFrame with 2 workers

In [ ]:
# Example data
df = pd.DataFrame({
    "note_id":  ["n1", "n2", "n3"],
    "note_text": [
        "Patient was arrested for DUI last year.",
        "Denies alcohol use; discharge instructions reviewed.",
        "History of opioid dependence; not currently using."
    ]
})

# Use packaged checklist + terms
checklist = sud.checklist_abc
termslist = sud.default_termslist

# Parallel on (2 workers) — your “n=2”
res = sud.extract_df(
    df=df,
    checklist=checklist,
    termslist=termslist,
    terms_active="alcohol_terms,opioid_terms",
    parallel=True,                # 👈 NEW: parallel on
    n_workers=2,                 # 👈 NEW: 2 workers
    include_note_text=True,
    exclude_discharge_mentions=False,  # default; set False to keep discharge-context hits
    negation_scope='left',
    debug=True,
)

res.head()


Using terms: ['alcohol', 'wine', 'beer', 'drink', 'drinking', 'drinks', 'ethanol', 'etoh', 'booze', 'cocktail', 'drunk', 'intoxicated', 'liquor', 'spirits', 'hard liquor', 'alcoholic beverage', 'bar', 'pub', 'alcoholism', 'alcohol abuse', 'fermented beverage', 'alcohol withdrawal', 'alcohol addiction', 'DUI (Driving Under the Influence)', 'alcoholic drink', 'alchol', 'alhcohol', 'alcol', 'alc-consumption', 'alcholic', 'alch', 'EtOH', 'alcahol', 'Et-OH', 'alcohol use', 'alcohol use disorder', 'pain med', 'opioid', 'opiod', 'narc', 'analges', 'suboxone', 'Avinza', 'codeine', 'dilaudid', 'fentanyl', 'hydrocodone', 'morphine', 'opana', 'opiate', 'oxycodone', 'oxycontin', 'oxymorphone', 'percocet', 'roxicodone', 'sufentanyl', 'vicodin', 'lortab', 'hydromorphone', 'abstral', 'actiq', 'alfentanil', 'arymo', 'ascomp', 'astramorph', 'avinza', 'belbuca', 'brompheniramine', 'bunavail', 'buprenex', 'buprenorphine', 'butalbital', 'butorphanol', 'butrans', 'capcof', 'carisoprodol', 'cheratussin', 'c

,note_id,illicit_drugs,illicit_drugs_SUBSTANCE_MATCHED,illicit_drugs_SUBSTANCE_MATCHED_NEG,problem_drinking,problem_drinking_NEG,dui,dui_NEG,hoarding,hoarding_SUBSTANCE_MATCHED,...,minimal_relief_x,minimal_relief_x_SUBSTANCE_MATCHED,tolerance,tolerance_SUBSTANCE_MATCHED,tolerance_SUBSTANCE_MATCHED_NEG,med_agreement,SO_concern,SO_concern_SUBSTANCE_MATCHED,SO_concern_SUBSTANCE_MATCHED_NEG,note_text
0,n1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,Patient was arrested for DUI last year.
1,n2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Denies alcohol use; discharge instructions rev...
2,n3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,History of opioid dependence; not currently us...



### 📋 Checklist Validation Summary
The following code can be used if you want to ensure that the checklist and termslist are working with your data. 
- ✅ Loads regex patterns from `snapshotchecklist.py`
- 📝 Parses clinical notes from `notes_input.txt`
- 🔍 Validates each note against its expected match
- 📄 Saves:
  - Full results to `package_validation_result.csv`
  - Checklist item level results `package_checklist_validation_by_item.csv`
- 📊 Returns both results as DataFrames for further use


In [ ]:
source_dir = "your_directory_here"  #Change this to your directory


detailed, by_item = sud.validation(
    checklist = dir + '/snapshotchecklist.py',
    examples = dir + '/notes_input.txt',
)

detailed

,item_code,expected,note_text,actual_match,mismatch
0,1a,1,Patient reports cocaine use last week.,1,0
1,1a,0,Patient reports no recreational substance use.,0,0
2,1a,0,Mother has history of depression.,0,0
3,1b,1,Problem drinking documented; alcohol abuse noted.,1,0
4,1b,0,Patient does not drink alcohol.,0,0
...,...,...,...,...,...
76,19,0,Patient follows the medication agreement witho...,0,0
77,19,0,Insurance agreement issue was noted.,0,0
78,20,1,Patient's wife is concerned about his use of p...,1,0
79,20,0,No relatives expressed alarm about medication ...,0,0


In [28]:
by_item

,item_key,n,correct,accuracy
0,10,3,3,1.0
1,11a,3,3,1.0
2,11b,3,3,1.0
3,12a,3,3,1.0
4,12b,3,3,1.0
5,13,3,3,1.0
6,14,3,3,1.0
7,15,3,3,1.0
8,16,3,3,1.0
9,17,3,3,1.0


# sudregex — Refactor Tutorial 🚀

This notebook walks you through the updated **gating utilities**, the new **negation scope**, and the **gated preview pipeline**. It also shows how to run both the **DataFrame API** and the **CLI** with the new options.

**Key changes covered:**
- Unified gates for **substance**, **negation**, **common false-positives**, **discharge contexts**
- `negation_scope` toggle: `'left' | 'right' | 'both'`
- Gated previews via `write_previews_for_item()` (only from the **final mask**)
- CLI & API passthrough flags for preview and negation options
- Terms list compatibility (`termslist.py` groups or raw `terms=[...]`)

> You can run this notebook inside your repo's virtual environment where `sudregex` is importable.

## 1) Imports
We'll import the main entry points you just refactored.

In [1]:
import pandas as pd
import os
import sudregex as sud
from pathlib import Path
print('sudregex version:', getattr(sud, '__version__', 'unknown'))

sudregex version: 0.1.2


## 3) Create a tiny demo dataset
We'll craft rows that exercise **positive**, **negated**, and **discharge** contexts, so you can see gated behavior & previews.

In [2]:
val_df = pd.DataFrame({
    'note_id': ['A1','A2','A3','A4','A5','A6'],
    'note_text': [
        'Patient reports cocaine use last week.',            # positive
        'Patient denies cocaine use.',                       # negated (left-side negation cue)
        'Discharge instructions: no cocaine use while...',  # discharge context (to be excluded if enabled)
        'Cocaine mentioned by family, unclear.',             # positive (no negation)
        'Testing negative for cocaine today.',               # negated via cue 'negative'
        'No illicit drug use. Cocaine not used.',            # strong negation left/right
    ]
})
val_df

,note_id,note_text
0,A1,Patient reports cocaine use last week.
1,A2,Patient denies cocaine use.
2,A3,Discharge instructions: no cocaine use while...
3,A4,"Cocaine mentioned by family, unclear."
4,A5,Testing negative for cocaine today.
5,A6,No illicit drug use. Cocaine not used.


## 4) Checklist (no substance terms, but with negation & previews)
This item searches for the exact word **cocaine** and applies **negation gating**, with previews turned on.

> Previews will be pulled **only from the final mask** after all gates (negation, discharge, etc.).

In [3]:
checklist = {
    'C1': {
        'lab': "C1 'Mentions of cocaine (exact word)'",
        'pat': r"\bcocaine\b",
        'substance': False,          # no proximity list required
        'negation': True,            # keep only non-negated mentions
        'col_name': 'cocaine_mention',
        'preview': True,             # gated previews enabled
        # optional: 'common_fp': [ ... ],
    }
}

## 5) Run `extract_df` with gated previews
We pass a **dummy** terms list (because this checklist doesn't use `substance=True`).

- `include_note_text=True` so the helper can build preview snippets.
- `preview_count=10` so we always write some previews if there are any kept rows.
- `negation_scope='left'` (default) — try `'both'` to tighten further.

In [5]:
out_preview_txt = 'note_previews.txt'
out_preview_csv = 'note_previews.csv'
if os.path.exists(out_preview_txt):
    os.remove(out_preview_txt)
if os.path.exists(out_preview_csv):
    os.remove(out_preview_csv)
    
result_df, previews_df = sud.extract_df(
    val_df,
    checklist=checklist,                 # dict or path
    terms=['__dummy__'],                 # or use termslist+terms_active
    note_column='note_text',
    id_column='note_id',
    include_note_text=True,              # optional; result_df will keep note_text if True
    exclude_discharge_mentions=True,
    preview_count=10,
    preview_span=120,
    # preview_file=out_preview_txt,      # optional: write human-readable file
    # preview_csv=out_preview_csv,       # optional: write CSV to disk
    negation_scope='left',
    debug=False,
    return_previews_df=True,             # <<< NEW: get previews as a DataFrame
)
# print('\nResult DataFrame:')
# print(result_df)
print('\nPreviews DataFrame:')
print(previews_df)




Previews DataFrame:
  item_key note_id  span_start  span_end  \
0       C1      A4           0         7   
1       C1      A1          16        23   

                                  snippet  \
0   Cocaine mentioned by family, unclear.   
1  Patient reports cocaine use last week.   

                               snippet_marked  
0   [[Cocaine]] mentioned by family, unclear.  
1  Patient reports [[cocaine]] use last week.  


/panfs/accrepfs.vampire/data/g_jeffery_lab/substance_use_disorders/code/preprocessing/regex/generalized/regex_package/sudregex/helper.py:815: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  counts = pd.concat([counts, df_to_search], sort=True)
/panfs/accrepfs.vampire/data/g_jeffery_lab/substance_use_disorders/code/preprocessing/regex/generalized/regex_package/sudregex/helper.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hits = df[df[in_col].fillna(0).astype(int) > 0].copy()


# 7)How to apply validation after refactoring

In [ ]:
df = pd.read_csv('/your_validation_examples.csv')
df.columns = ['item_key', 'expected', 'note_text']

detailed, by_item = sud.validation(
    checklist= "/your_checklist.py",
    examples = df
    
    )

# detailed
by_item

,item_key,n,tp,fp,fn,precision,recall,f1
0,C1,19,6,4,1,0.6,0.857143,0.705882
1,TOTAL,19,6,4,1,0.6,0.857143,0.705882


## 8) CLI usage
Equivalent CLI run for a CSV input:
```bash
sudregex \
  --extract \
  --checklist /path/to/checklist.py \
  --in_file /path/to/input.csv \
  --out_file /path/to/output.csv \
  --separator , \
  --include_note_text \
  --preview-count 10 \
  --preview-span 120 \
  --preview-file ./note_previews.txt \
  --preview-csv  ./note_previews.csv \
  --negation-scope left \
  --exclude-discharge-mentions \
  --terms __dummy__
```
If using a `termslist.py`:
```bash
sudregex \
  --extract \
  --checklist /path/to/checklist.py \
  --in_file /path/to/input.csv \
  --out_file /path/to/output.csv \
  --termslist /path/to/termslist.py \
  --terms_active opioid_terms,alcohol_terms \
  --include_note_text --preview-count 10 --preview-span 120 \
  --preview-file ./note_previews.txt --preview-csv ./note_previews.csv \
  --negation-scope both --exclude-discharge-mentions
```


## 9) FAQ / Tips
- If your checklist doesn't use `substance=True`, you can pass a dummy list via `terms=['__dummy__']`.
- Try `negation_scope='both'` to be stricter about negations.
- Set `preview_count=0` to disable previews entirely.
- If you see missing columns in merges, enable `debug=True` and read the `[DEBUG]` lines to follow the gating path.
